# 📚 CPV301 Complete Image Processing Pipeline
## Master Pipeline for Practical Exam - All Topics Covered

This notebook contains a comprehensive pipeline that processes an input image through ALL the topics covered in CPV301:

### 🎯 Pipeline Coverage:
1. **Image Loading & Basic Operations**
2. **Color Space Conversions** 
3. **Point Operators** (Histogram Equalization, Thresholding, Contrast Stretching)
4. **Image Smoothing & Filtering**
5. **Edge Detection** (Gradients, Canny)
6. **Morphological Transformations**
7. **Geometric Transformations**
8. **White Balancing**
9. **Feature Detection** (Harris, FAST, SIFT)
10. **Fourier Transform**

### 📝 Instructions:
- Replace the image path in the first code cell with your test image
- Run all cells to see the complete processing pipeline
- Each section shows theory, implementation, and results
- Perfect for exam preparation and understanding!

## 🚀 Essential Imports and Helper Functions

In [ ]:
# Essential imports for all image processing operations
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib parameters for better visualization
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

def show_images(images, titles, rows=2, cols=3, figsize=(15, 10), cmap_list=None):
    """
    Enhanced function to display multiple images in a grid
    
    Args:
        images: List of images to display
        titles: List of titles for each image
        rows, cols: Grid dimensions
        figsize: Figure size
        cmap_list: List of colormaps for each image (optional)
    """
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.flatten() if rows * cols > 1 else [axes]
    
    for i, (img, title) in enumerate(zip(images, titles)):
        if i >= len(axes):
            break
            
        ax = axes[i]
        
        # Determine colormap
        if cmap_list and i < len(cmap_list):
            cmap = cmap_list[i]
        elif len(img.shape) == 2:
            cmap = 'gray'
        else:
            cmap = None
            
        # Handle BGR to RGB conversion for color images
        if len(img.shape) == 3 and cmap is None:
            display_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        else:
            display_img = img
            
        ax.imshow(display_img, cmap=cmap)
        ax.set_title(title, fontsize=12, fontweight='bold')
        ax.axis('off')
    
    # Hide remaining subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

def create_test_image():
    """Create a test image if no real image is available"""
    # Create a synthetic test image with various features
    img = np.zeros((400, 400, 3), dtype=np.uint8)
    
    # Add colored rectangles
    cv2.rectangle(img, (50, 50), (150, 150), (255, 100, 100), -1)
    cv2.rectangle(img, (200, 50), (350, 150), (100, 255, 100), -1)
    cv2.rectangle(img, (50, 200), (150, 350), (100, 100, 255), -1)
    
    # Add circles
    cv2.circle(img, (275, 275), 50, (255, 255, 100), -1)
    cv2.circle(img, (125, 275), 30, (255, 150, 200), -1)
    
    # Add some noise
    noise = np.random.randint(0, 50, img.shape, dtype=np.uint8)
    img = cv2.add(img, noise)
    
    return img

def debug_image(img, title="Debug Image"):
    """Quick debugging function to check image properties"""
    print(f"=== {title} ===")
    print(f"Shape: {img.shape}")
    print(f"Type: {img.dtype}")
    print(f"Min: {img.min()}, Max: {img.max()}")
    print(f"Mean: {img.mean():.2f}")
    print("-" * 30)

print("✅ Helper functions loaded successfully!")
print("📝 Ready to process images through the complete pipeline!")

## 📖 1. Image Loading and Basic Operations
### Load your image here and perform basic operations

In [ ]:
# ========================================
# CHANGE THIS PATH TO YOUR TEST IMAGE
# ========================================
IMAGE_PATH = 'D:/FPT_Material/Sem 4/CPV301/Source for PE/Image/image.jpg'

# Try to load the specified image, fallback to test image if not found
try:
    original_img = cv2.imread(IMAGE_PATH)
    if original_img is None:
        raise FileNotFoundError("Image not found")
    print(f"✅ Successfully loaded image from: {IMAGE_PATH}")
except:
    print("⚠️ Could not load specified image, creating test image...")
    original_img = create_test_image()

# Get image properties
debug_image(original_img, "Original Image")

# Convert to different basic formats
rgb_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
gray_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)

# Display basic formats
show_images(
    [rgb_img, gray_img],
    ['Original Image (RGB)', 'Grayscale Version'],
    rows=1, cols=2, figsize=(12, 5)
)

## 🎨 2. Color Space Conversions
### Converting between different color representations

In [ ]:
# Convert to various color spaces
hsv_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2HSV)
lab_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2LAB)
ycrcb_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2YCrCb)
luv_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2LUV)

# For proper display, convert back to RGB
hsv_display = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2RGB)
lab_display = cv2.cvtColor(lab_img, cv2.COLOR_LAB2RGB)
ycrcb_display = cv2.cvtColor(ycrcb_img, cv2.COLOR_YCrCb2RGB)
luv_display = cv2.cvtColor(luv_img, cv2.COLOR_LUV2RGB)

print("🎨 Color Space Analysis:")
print(f"HSV range - H: [0,179], S: [0,255], V: [0,255]")
print(f"LAB range - L: [0,100], A: [-127,127], B: [-127,127]")
print(f"YCrCb range - Y: [16,235], Cr: [16,240], Cb: [16,240]")

show_images(
    [rgb_img, hsv_display, lab_display, ycrcb_display, luv_display, gray_img],
    ['Original RGB', 'HSV Color Space', 'LAB Color Space', 'YCrCb Color Space', 'LUV Color Space', 'Grayscale'],
    rows=2, cols=3
)

## 🔧 3. Point Operators
### Pixel-wise transformations: Histogram Equalization, Thresholding, Contrast Enhancement

In [ ]:
# ========================================
# 3.1 Histogram Equalization
# ========================================
print("📊 Applying Histogram Equalization...")
equalized_img = cv2.equalizeHist(gray_img)

# ========================================
# 3.2 Contrast Stretching (Normalization)
# ========================================
print("📈 Applying Contrast Stretching...")
stretched_img = cv2.normalize(gray_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

# ========================================
# 3.3 Alpha Compositing Example
# ========================================
print("🎭 Creating Alpha Compositing Example...")
def alpha_compositing(foreground, background, alpha_matte):
    """Alpha compositing: C = αF + (1-α)B"""
    alpha = alpha_matte.astype(float) / 255.0
    if len(alpha.shape) == 2:
        alpha = np.repeat(alpha[:, :, np.newaxis], 3, axis=2)
    
    foreground = foreground.astype(float)
    background = background.astype(float)
    composite = alpha * foreground + (1 - alpha) * background
    return np.clip(composite, 0, 255).astype(np.uint8)

# Create simple alpha matte (circular gradient)
h, w = gray_img.shape
center_x, center_y = w//2, h//2
Y, X = np.ogrid[:h, :w]
distances = np.sqrt((X - center_x)**2 + (Y - center_y)**2)
alpha_matte = np.clip(255 - distances * 2, 0, 255).astype(np.uint8)

# Create a simple background (solid color)
background = np.full_like(original_img, [100, 150, 200])  # Light blue background
composite_result = alpha_compositing(original_img, background, alpha_matte)

# Display point operations results
show_images(
    [gray_img, equalized_img, stretched_img, alpha_matte, cv2.cvtColor(composite_result, cv2.COLOR_BGR2RGB)],
    ['Original Grayscale', 'Histogram Equalized', 'Contrast Stretched', 'Alpha Matte', 'Alpha Compositing'],
    rows=2, cols=3
)

# Show histograms
plt.figure(figsize=(15, 4))

plt.subplot(1, 3, 1)
plt.hist(gray_img.ravel(), 256, [0, 256], color='blue', alpha=0.7)
plt.title('Original Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(equalized_img.ravel(), 256, [0, 256], color='green', alpha=0.7)
plt.title('Equalized Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.hist(stretched_img.ravel(), 256, [0, 256], color='red', alpha=0.7)
plt.title('Stretched Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## 🔒 4. Thresholding Operations
### Binary image creation using different thresholding techniques

In [ ]:
# ========================================
# 4.1 Simple Thresholding
# ========================================
print("🔒 Applying Different Thresholding Techniques...")

ret1, thresh_binary = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY)
ret2, thresh_binary_inv = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY_INV)
ret3, thresh_trunc = cv2.threshold(gray_img, 127, 255, cv2.THRESH_TRUNC)
ret4, thresh_tozero = cv2.threshold(gray_img, 127, 255, cv2.THRESH_TOZERO)
ret5, thresh_tozero_inv = cv2.threshold(gray_img, 127, 255, cv2.THRESH_TOZERO_INV)

# ========================================
# 4.2 Adaptive Thresholding
# ========================================
# Apply slight blur to reduce noise
blurred_for_adaptive = cv2.medianBlur(gray_img, 5)

adaptive_mean = cv2.adaptiveThreshold(blurred_for_adaptive, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                     cv2.THRESH_BINARY, 11, 2)
adaptive_gaussian = cv2.adaptiveThreshold(blurred_for_adaptive, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                         cv2.THRESH_BINARY, 11, 2)

# ========================================
# 4.3 Otsu's Thresholding
# ========================================
ret_otsu, otsu_thresh = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Also try Otsu with Gaussian filtering
gaussian_blur = cv2.GaussianBlur(gray_img, (5, 5), 0)
ret_otsu_blur, otsu_thresh_blur = cv2.threshold(gaussian_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

print(f"📊 Thresholding Results:")
print(f"Simple threshold value: {ret1}")
print(f"Otsu threshold value: {ret_otsu:.1f}")
print(f"Otsu with blur threshold value: {ret_otsu_blur:.1f}")

# Display all thresholding results
show_images(
    [gray_img, thresh_binary, thresh_binary_inv, thresh_trunc, thresh_tozero, thresh_tozero_inv],
    ['Original', 'Binary', 'Binary Inverted', 'Truncated', 'To Zero', 'To Zero Inverted'],
    rows=2, cols=3
)

show_images(
    [gray_img, adaptive_mean, adaptive_gaussian, otsu_thresh, otsu_thresh_blur],
    ['Original', 'Adaptive Mean', 'Adaptive Gaussian', 'Otsu', 'Otsu + Blur'],
    rows=2, cols=3
)

## 🌊 5. Image Smoothing and Filtering
### Noise reduction and smoothing using different filter types

In [ ]:
# ========================================
# 5.1 Linear Filters
# ========================================
print("🌊 Applying Various Smoothing Filters...")

# Averaging filter
avg_blur = cv2.blur(original_img, (9, 9))

# Gaussian filter
gaussian_blur = cv2.GaussianBlur(original_img, (9, 9), 0)

# Box filter (normalized and unnormalized)
box_filter_norm = cv2.boxFilter(original_img, -1, (9, 9), normalize=True)
box_filter_unnorm = cv2.boxFilter(original_img, -1, (9, 9), normalize=False)

# ========================================
# 5.2 Non-linear Filters
# ========================================
# Median filter (excellent for salt-and-pepper noise)
median_blur = cv2.medianBlur(original_img, 9)

# Bilateral filter (edge-preserving)
bilateral_filter = cv2.bilateralFilter(original_img, 9, 75, 75)

# ========================================
# 5.3 Custom Kernel Filtering
# ========================================
# Create custom sharpening kernel
sharpen_kernel = np.array([[-1, -1, -1],
                          [-1,  9, -1],
                          [-1, -1, -1]])
sharpened = cv2.filter2D(original_img, -1, sharpen_kernel)

print("📊 Filter Comparison:")
print("- Averaging: Simple blur, may create artifacts")
print("- Gaussian: Smooth blur, good for noise reduction")
print("- Median: Excellent for salt-and-pepper noise")
print("- Bilateral: Preserves edges while smoothing")

show_images(
    [cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB), 
     cv2.cvtColor(avg_blur, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(gaussian_blur, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(median_blur, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(bilateral_filter, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(sharpened, cv2.COLOR_BGR2RGB)],
    ['Original', 'Average Blur', 'Gaussian Blur', 'Median Blur', 'Bilateral Filter', 'Sharpened'],
    rows=2, cols=3
)

## ⚡ 6. Edge Detection and Gradients
### Finding edges using gradient operators and Canny edge detection

In [ ]:
# ========================================
# 6.1 Gradient Operators
# ========================================
print("⚡ Applying Gradient Operators and Edge Detection...")

# IMPORTANT: Always use CV_64F for gradients to capture negative values!
sobel_x = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=5)
sobel_y = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=5)

# Scharr operator (more accurate than 3x3 Sobel)
scharr_x = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=-1)  # ksize=-1 for Scharr
scharr_y = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=-1)

# Laplacian operator (second derivative)
laplacian = cv2.Laplacian(gray_img, cv2.CV_64F)

# Convert back to uint8 for display
sobel_x_8u = np.uint8(np.absolute(sobel_x))
sobel_y_8u = np.uint8(np.absolute(sobel_y))
scharr_x_8u = np.uint8(np.absolute(scharr_x))
scharr_y_8u = np.uint8(np.absolute(scharr_y))
laplacian_8u = np.uint8(np.absolute(laplacian))

# Combine gradients
sobel_combined = np.uint8(np.sqrt(sobel_x**2 + sobel_y**2))
scharr_combined = np.uint8(np.sqrt(scharr_x**2 + scharr_y**2))

# ========================================
# 6.2 Canny Edge Detection
# ========================================
# Preprocess with Gaussian blur
preprocessed = cv2.GaussianBlur(gray_img, (5, 5), 0)

# Apply Canny with different threshold combinations
canny_low = cv2.Canny(preprocessed, 50, 100)
canny_medium = cv2.Canny(preprocessed, 100, 200)
canny_high = cv2.Canny(preprocessed, 150, 300)

print("📊 Edge Detection Analysis:")
print(f"Sobel X gradient range: [{sobel_x.min():.1f}, {sobel_x.max():.1f}]")
print(f"Sobel Y gradient range: [{sobel_y.min():.1f}, {sobel_y.max():.1f}]")
print(f"Laplacian range: [{laplacian.min():.1f}, {laplacian.max():.1f}]")

# Display gradient results
show_images(
    [gray_img, sobel_x_8u, sobel_y_8u, sobel_combined, scharr_combined, laplacian_8u],
    ['Original', 'Sobel X', 'Sobel Y', 'Sobel Combined', 'Scharr Combined', 'Laplacian'],
    rows=2, cols=3
)

# Display Canny results
show_images(
    [gray_img, canny_low, canny_medium, canny_high],
    ['Original', 'Canny (50,100)', 'Canny (100,200)', 'Canny (150,300)'],
    rows=2, cols=2
)

## 🔧 7. Morphological Transformations
### Shape-based operations for binary image processing

In [ ]:
# ========================================
# 7.1 Basic Morphological Operations
# ========================================
print("🔧 Applying Morphological Transformations...")

# Use binary image from thresholding
binary_img = thresh_binary.copy()

# Define different kernels
kernel_3x3 = np.ones((3, 3), np.uint8)
kernel_5x5 = np.ones((5, 5), np.uint8)
kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
kernel_cross = cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5))

# Basic operations
erosion = cv2.erode(binary_img, kernel_5x5, iterations=1)
dilation = cv2.dilate(binary_img, kernel_5x5, iterations=1)

# Compound operations
opening = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernel_5x5)
closing = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel_5x5)
morph_gradient = cv2.morphologyEx(binary_img, cv2.MORPH_GRADIENT, kernel_5x5)
tophat = cv2.morphologyEx(binary_img, cv2.MORPH_TOPHAT, kernel_5x5)
blackhat = cv2.morphologyEx(binary_img, cv2.MORPH_BLACKHAT, kernel_5x5)

print("📊 Morphological Operations Effects:")
print("- Erosion: Shrinks foreground objects")
print("- Dilation: Expands foreground objects")
print("- Opening: Removes small objects, separates connected objects")
print("- Closing: Fills small holes, connects nearby objects")
print("- Gradient: Highlights boundaries")
print("- Top Hat: Enhances bright details on dark background")
print("- Black Hat: Enhances dark details on bright background")

show_images(
    [binary_img, erosion, dilation, opening, closing, morph_gradient],
    ['Original Binary', 'Erosion', 'Dilation', 'Opening', 'Closing', 'Gradient'],
    rows=2, cols=3
)

show_images(
    [binary_img, tophat, blackhat],
    ['Original Binary', 'Top Hat', 'Black Hat'],
    rows=1, cols=3
)

## 🔄 8. Geometric Transformations
### Spatial transformations: Translation, Rotation, Scaling, Affine, Perspective

In [ ]:
# ========================================
# 8.1 Basic Geometric Transformations
# ========================================
print("🔄 Applying Geometric Transformations...")

rows, cols = gray_img.shape
center = (cols // 2, rows // 2)

# ========================================
# Translation
# ========================================
tx, ty = 50, 30  # Translation offsets
M_translate = np.float32([[1, 0, tx], [0, 1, ty]])
translated = cv2.warpAffine(gray_img, M_translate, (cols, rows))

# ========================================
# Rotation (Euclidean)
# ========================================
angle = 45  # degrees
scale = 1.0
M_rotate = cv2.getRotationMatrix2D(center, angle, scale)
rotated = cv2.warpAffine(gray_img, M_rotate, (cols, rows))

# ========================================
# Similarity Transformation (Rotation + Scaling)
# ========================================
angle_sim = 30
scale_sim = 0.8
M_similarity = cv2.getRotationMatrix2D(center, angle_sim, scale_sim)
similarity = cv2.warpAffine(gray_img, M_similarity, (cols, rows))

# ========================================
# Affine Transformation
# ========================================
# Define 3 point correspondences
pts1 = np.float32([[50, 50], [200, 50], [50, 200]])
pts2 = np.float32([[10, 100], [200, 50], [100, 250]])

# Scale points if image is too small
if rows < 300 or cols < 300:
    scale_factor = min(rows, cols) / 300
    pts1 = pts1 * scale_factor
    pts2 = pts2 * scale_factor

M_affine = cv2.getAffineTransform(pts1, pts2)
affine = cv2.warpAffine(gray_img, M_affine, (cols, rows))

print("📊 Transformation Hierarchy:")
print("Translation (2 DOF) → Euclidean (3 DOF) → Similarity (4 DOF) → Affine (6 DOF) → Perspective (8 DOF)")
print(f"Original image size: {rows}x{cols}")
print(f"Translation offset: ({tx}, {ty})")
print(f"Rotation angle: {angle}°")
print(f"Similarity: {angle_sim}° rotation, {scale_sim}x scale")

show_images(
    [gray_img, translated, rotated, similarity, affine],
    ['Original', 'Translated', 'Rotated', 'Similarity', 'Affine'],
    rows=2, cols=3
)

## ⚖️ 9. White Balancing
### Color correction algorithms for proper color representation

In [ ]:
# ========================================
# 9.1 Gray World White Balancing
# ========================================
print("⚖️ Applying White Balancing Algorithms...")

def gray_world_white_balance(img):
    """Gray World Algorithm: Assumes the average color should be gray"""
    # Calculate channel averages
    avg_b = np.mean(img[:, :, 0])
    avg_g = np.mean(img[:, :, 1]) 
    avg_r = np.mean(img[:, :, 2])
    
    # Overall average
    avg = (avg_b + avg_g + avg_r) / 3
    
    # Scaling factors
    scale_b = avg / avg_b if avg_b > 0 else 1
    scale_g = avg / avg_g if avg_g > 0 else 1
    scale_r = avg / avg_r if avg_r > 0 else 1
    
    # Apply scaling
    balanced = np.zeros_like(img, dtype=np.float32)
    balanced[:, :, 0] = img[:, :, 0] * scale_b
    balanced[:, :, 1] = img[:, :, 1] * scale_g
    balanced[:, :, 2] = img[:, :, 2] * scale_r
    
    return np.clip(balanced, 0, 255).astype(np.uint8)

def white_patch_white_balance(img, percentile=99):
    """White Patch Algorithm: Assumes brightest pixels should be white"""
    # Find percentile values in each channel
    max_b = np.percentile(img[:, :, 0], percentile)
    max_g = np.percentile(img[:, :, 1], percentile)
    max_r = np.percentile(img[:, :, 2], percentile)
    
    # Target value
    max_val = 250.0
    
    # Scaling factors
    scale_b = max_val / max_b if max_b > 0 else 1
    scale_g = max_val / max_g if max_g > 0 else 1
    scale_r = max_val / max_r if max_r > 0 else 1
    
    # Apply scaling
    balanced = np.zeros_like(img, dtype=np.float32)
    balanced[:, :, 0] = np.clip(img[:, :, 0] * scale_b, 0, 255)
    balanced[:, :, 1] = np.clip(img[:, :, 1] * scale_g, 0, 255)
    balanced[:, :, 2] = np.clip(img[:, :, 2] * scale_r, 0, 255)
    
    return balanced.astype(np.uint8)

# Apply different white balancing algorithms
gray_world_result = gray_world_white_balance(original_img)
white_patch_result = white_patch_white_balance(original_img)

# Analyze color statistics
def analyze_color_stats(img, name):
    avg_b, avg_g, avg_r = np.mean(img, axis=(0,1))
    print(f"{name}: B={avg_b:.1f}, G={avg_g:.1f}, R={avg_r:.1f}")

print("📊 Color Statistics Analysis:")
analyze_color_stats(original_img, "Original")
analyze_color_stats(gray_world_result, "Gray World")
analyze_color_stats(white_patch_result, "White Patch")

show_images(
    [cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(gray_world_result, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(white_patch_result, cv2.COLOR_BGR2RGB)],
    ['Original Image', 'Gray World WB', 'White Patch WB'],
    rows=1, cols=3
)

## 🎯 10. Feature Detection
### Detecting and describing key points: Harris, FAST, SIFT, ORB

In [ ]:
# ========================================
# 10.1 Harris Corner Detection
# ========================================
print("🎯 Applying Feature Detection Algorithms...")

# Harris corner detection
gray_float = np.float32(gray_img)
harris_corners = cv2.cornerHarris(gray_float, 2, 3, 0.04)

# Create visualization
harris_img = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
harris_img[harris_corners > 0.01 * harris_corners.max()] = [0, 0, 255]  # Mark corners in red

# ========================================
# 10.2 FAST Corner Detection
# ========================================
fast = cv2.FastFeatureDetector_create()
fast_keypoints = fast.detect(gray_img, None)
fast_img = cv2.drawKeypoints(cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR), fast_keypoints, None, color=(0,255,0))

# ========================================
# 10.3 SIFT Features (if available)
# ========================================
try:
    sift = cv2.SIFT_create()
    sift_keypoints, sift_descriptors = sift.detectAndCompute(gray_img, None)
    sift_img = cv2.drawKeypoints(cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR), sift_keypoints, None)
    sift_available = True
except:
    print("⚠️ SIFT not available, using ORB instead...")
    sift_img = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
    sift_keypoints = []
    sift_available = False

# ========================================
# 10.4 ORB Features (Alternative to SIFT/SURF)
# ========================================
orb = cv2.ORB_create()
orb_keypoints, orb_descriptors = orb.detectAndCompute(gray_img, None)
orb_img = cv2.drawKeypoints(cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR), orb_keypoints, None, color=(255,0,0))

# ========================================
# 10.5 Good Features to Track
# ========================================
corners = cv2.goodFeaturesToTrack(gray_img, 100, 0.01, 10)
corners_img = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)

if corners is not None:
    corners = np.int0(corners)
    for corner in corners:
        x, y = corner.ravel()
        cv2.circle(corners_img, (x, y), 3, (255, 255, 0), -1)

print(f"📊 Feature Detection Results:")
print(f"Harris corners found: {np.sum(harris_corners > 0.01 * harris_corners.max())}")
print(f"FAST keypoints: {len(fast_keypoints)}")
if sift_available:
    print(f"SIFT keypoints: {len(sift_keypoints)}")
print(f"ORB keypoints: {len(orb_keypoints)}")
print(f"Good features to track: {len(corners) if corners is not None else 0}")

show_images(
    [cv2.cvtColor(harris_img, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(fast_img, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(sift_img, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(orb_img, cv2.COLOR_BGR2RGB),
     cv2.cvtColor(corners_img, cv2.COLOR_BGR2RGB)],
    ['Harris Corners', 'FAST Features', 'SIFT Features', 'ORB Features', 'Good Features to Track'],
    rows=2, cols=3
)

## 🌊 11. Fourier Transform
### Frequency domain analysis and filtering

In [ ]:
# ========================================
# 11.1 Forward Fourier Transform
# ========================================
print("🌊 Applying Fourier Transform Analysis...")

# Apply FFT
f_transform = np.fft.fft2(gray_img)
f_shift = np.fft.fftshift(f_transform)

# Calculate magnitude spectrum for visualization
magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1)  # +1 to avoid log(0)

# ========================================
# 11.2 Frequency Domain Filtering
# ========================================
rows, cols = gray_img.shape
crow, ccol = rows // 2, cols // 2

# Low-pass filter (removes high frequencies)
mask_low = np.zeros((rows, cols), np.uint8)
cv2.circle(mask_low, (ccol, crow), 50, 1, -1)

# Apply low-pass filter
fshift_low = f_shift * mask_low
f_ishift_low = np.fft.ifftshift(fshift_low)
img_back_low = np.fft.ifft2(f_ishift_low)
img_back_low = np.abs(img_back_low)

# High-pass filter (removes low frequencies)
mask_high = np.ones((rows, cols), np.uint8)
cv2.circle(mask_high, (ccol, crow), 50, 0, -1)

# Apply high-pass filter
fshift_high = f_shift * mask_high
f_ishift_high = np.fft.ifftshift(fshift_high)
img_back_high = np.fft.ifft2(f_ishift_high)
img_back_high = np.abs(img_back_high)

print("📊 Frequency Domain Analysis:")
print(f"Image size: {rows}x{cols}")
print(f"Center frequency: ({crow}, {ccol})")
print(f"Low-pass cutoff radius: 50 pixels")
print(f"High-pass cutoff radius: 50 pixels")

# Display frequency domain results
show_images(
    [gray_img, magnitude_spectrum, mask_low*255, img_back_low.astype(np.uint8), 
     mask_high*255, img_back_high.astype(np.uint8)],
    ['Original', 'Magnitude Spectrum', 'Low-pass Mask', 'Low-pass Result', 
     'High-pass Mask', 'High-pass Result'],
    rows=2, cols=3
)

## 📊 12. Pipeline Summary and Analysis
### Final comparison and exam preparation summary

In [ ]:
# ========================================
# 12.1 Pipeline Summary
# ========================================
print("📊 Complete Pipeline Analysis Summary")
print("=" * 60)

# Collect key results for final comparison
final_results = [
    ('Original Image', cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)),
    ('Grayscale', gray_img),
    ('Histogram Equalized', equalized_img),
    ('Gaussian Blur', cv2.cvtColor(gaussian_blur, cv2.COLOR_BGR2RGB)),
    ('Canny Edges', canny_medium),
    ('Binary Threshold', thresh_binary),
    ('Morphological Opening', opening),
    ('Harris Corners', cv2.cvtColor(harris_img, cv2.COLOR_BGR2RGB)),
    ('ORB Features', cv2.cvtColor(orb_img, cv2.COLOR_BGR2RGB)),
    ('Rotated', rotated),
    ('White Balanced', cv2.cvtColor(gray_world_result, cv2.COLOR_BGR2RGB)),
    ('Frequency Filtered', img_back_low.astype(np.uint8))
]

# Create final comparison grid
show_images(
    [result[1] for result in final_results],
    [result[0] for result in final_results],
    rows=3, cols=4, figsize=(20, 15)
)

# ========================================
# 12.2 Exam Tips Summary
# ========================================
print(f"\n🎯 CPV301 Practical Exam Tips:")
print("=" * 40)
print("1. Always check image loading: assert img is not None")
print("2. Use CV_64F for gradient operations to capture negative values")
print("3. Convert BGR→RGB for matplotlib display")
print("4. Choose appropriate kernel sizes (odd numbers)")
print("5. Apply preprocessing (blur) before edge detection")
print("6. Use adaptive thresholding for non-uniform lighting")
print("7. Combine morphological operations for better results")
print("8. Test different parameter values and compare results")
print("9. Understand when to use each technique based on image characteristics")
print("10. Always validate results visually and quantitatively")

print(f"\n✅ Pipeline completed successfully!")
print(f"📚 All CPV301 topics covered and demonstrated!")
print(f"💪 You're ready for the practical exam!")